In [9]:
import pandas as pd
import numpy as np
# использую модуль pandasql для написания sql запросов в питоне
from pandasql import sqldf

<font size = 5>**Задание** </font>

In [2]:
from IPython.display import Image
Image(url='images/2023-11-06_15-46-33.png')

In [3]:
Image(url='images/2023-11-06_15-46-52.png')

In [4]:
Image(url='images/2023-11-06_15-46-42.png')

Создаю тестовые датасеты

In [5]:
data_manager = pd.DataFrame({'id':[1,2,3,4,5,6,7,8,9,10,11,12,13,14],
                             'manager_name':['Никита','Маша','Вячеслав','Магнус','Сельдерей','Петя','Валя','Коля','Настя','Виталий','Рулон','Олег','Роман','Дима'],
                           'sal':[120,130,10,130,50,50,40,40,70,10,120,111,40,10]})

data_vsp = pd.DataFrame({'id':[1,2],
                        'vsp_name':['Бутово','Аэропорт']})

map_vsp2manager = pd.DataFrame({'vsp_id':[1,1,1,1,1,1,1,2,2,2,2,2,2,2],
                                'manager_id':[1,2,3,4,5,6,7,8,9,10,11,12,13,14]})

In [6]:
data_manager.head(5)

,id,manager_name,sal
0,1,Никита,120
1,2,Маша,130
2,3,Вячеслав,10
3,4,Магнус,130
4,5,Сельдерей,50


In [7]:
data_vsp

,id,vsp_name
0,1,Бутово
1,2,Аэропорт


In [8]:
map_vsp2manager.head(5)

,vsp_id,manager_id
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


<font size = 5>**Решение** </font>

**Способ 1. Через коррелирующий подзапрос**

In [31]:
q = """
WITH t1 as     
    (SELECT manager_name,
        sal,
        manager_id,
        vsp_name,
        DENSE_RANK () OVER (PARTITION BY data_vsp.vsp_name ORDER BY sal DESC) as rank
    FROM data_manager
    LEFT JOIN map_vsp2manager
    ON data_manager.id = map_vsp2manager.manager_id
    LEFT JOIN data_vsp
    ON map_vsp2manager.vsp_id = data_vsp.id)
    
SELECT manager_name,
        sal,
        vsp_name,
        CASE 
        WHEN sal = MAX(sal) OVER (PARTITION BY vsp_name) THEN '-1'
        ELSE (SELECT manager_name 
            FROM t1 as a 
            WHERE a.vsp_name = b.vsp_name and a.rank = b.rank-1 
            ORDER BY a.manager_id DESC 
            LIMIT 1)
        END as bigger_sal_name,
        CASE 
        WHEN sal = MAX(sal) OVER (PARTITION BY vsp_name) THEN -1
        ELSE (SELECT sal 
            FROM t1 as a 
            WHERE a.vsp_name = b.vsp_name and a.rank = b.rank-1 
            ORDER BY a.manager_id DESC 
            LIMIT 1)
        END as bigger_sal
FROM t1 as b
ORDER BY vsp_name, sal DESC, manager_id DESC"""
sqldf(q)

,manager_name,sal,vsp_name,bigger_sal_name,bigger_sal
0,Рулон,120,Аэропорт,-1,-1
1,Олег,111,Аэропорт,Рулон,120
2,Настя,70,Аэропорт,Олег,111
3,Роман,40,Аэропорт,Настя,70
4,Коля,40,Аэропорт,Настя,70
5,Дима,10,Аэропорт,Роман,40
6,Виталий,10,Аэропорт,Роман,40
7,Магнус,130,Бутово,-1,-1
8,Маша,130,Бутово,-1,-1
9,Никита,120,Бутово,Магнус,130


**Способ 2. Через self join**

In [16]:
q = """
WITH t1 as     
    (SELECT manager_name,
        sal,
        manager_id,
        vsp_name,
        DENSE_RANK () OVER (PARTITION BY data_vsp.vsp_name ORDER BY sal DESC) as rank,
        DENSE_RANK () OVER (PARTITION BY data_vsp.vsp_name ORDER BY sal DESC) + 1 as coef
    FROM data_manager
    LEFT JOIN map_vsp2manager
    ON data_manager.id = map_vsp2manager.manager_id
    LEFT JOIN data_vsp
    ON map_vsp2manager.vsp_id = data_vsp.id),
    
 t2 as (SELECT *
 FROM
 (SELECT DISTINCT manager_name, sal, vsp_name, 
     MAX(manager_id) OVER (PARTITION BY manager_name, sal, vsp_name ORDER BY sal DESC) as id 
 FROM
(SELECT a.manager_name,
        a.sal,
        a.vsp_name,
        b.manager_id,
        COALESCE(b.vsp_name, a.vsp_name) as bigger_vsp
    FROM t1 as a
    LEFT JOIN t1 as b
    ON a.rank = b.coef) as c
WHERE vsp_name = bigger_vsp) as d
ORDER BY vsp_name, sal DESC)

SELECT t2.manager_name,
    t2.sal,
    t2.vsp_name,
    COALESCE(t3.manager_name, '-1') as bigger_sal_name,
    COALESCE(t3.sal, '-1') as bigger_sal
FROM t2
LEFT JOIN data_manager as t3
USING (id)
"""
sqldf(q)

,manager_name,sal,vsp_name,bigger_sal_name,bigger_sal
0,Рулон,120,Аэропорт,-1,-1
1,Олег,111,Аэропорт,Рулон,120
2,Настя,70,Аэропорт,Олег,111
3,Коля,40,Аэропорт,Настя,70
4,Роман,40,Аэропорт,Настя,70
5,Виталий,10,Аэропорт,Роман,40
6,Дима,10,Аэропорт,Роман,40
7,Магнус,130,Бутово,-1,-1
8,Маша,130,Бутово,-1,-1
9,Никита,120,Бутово,Магнус,130


Еще один способ, но смердижная таблица уже хранится в датасете df

In [55]:
q = """
SELECT manager_name,
        sal,
        map_vsp2manager.vsp_id as vsp_id,
        map_vsp2manager.manager_id as manager_id,
        data_vsp.vsp_name as vsp_name,
        DENSE_RANK () OVER (PARTITION BY data_vsp.vsp_name ORDER BY sal DESC) as rank
    FROM data_manager
    LEFT JOIN map_vsp2manager
    ON data_manager.id = map_vsp2manager.manager_id
    LEFT JOIN data_vsp
    ON map_vsp2manager.vsp_id = data_vsp.id
"""
sqldf(q)

,manager_name,sal,vsp_id,manager_id,vsp_name,rank
0,Рулон,120,2,11,Аэропорт,1
1,Олег,111,2,12,Аэропорт,2
2,Настя,70,2,9,Аэропорт,3
3,Коля,40,2,8,Аэропорт,4
4,Роман,40,2,13,Аэропорт,4
5,Виталий,10,2,10,Аэропорт,5
6,Дима,10,2,14,Аэропорт,5
7,Маша,130,1,2,Бутово,1
8,Магнус,130,1,4,Бутово,1
9,Никита,120,1,1,Бутово,2


In [56]:
df = sqldf(q)

In [14]:
q = """    
SELECT manager_name,
        manager_id,
        sal,
        vsp_name,
        rank,
        CASE 
        WHEN sal = MAX(sal) OVER (PARTITION BY vsp_name) THEN '-1'
        ELSE (
            SELECT t2.manager_name
            FROM df AS t2
            WHERE t2.vsp_name = t1.vsp_name
                AND t2.rank = t1.rank - 1
                AND (
                    t2.sal > t1.sal
                    OR (t2.sal = t1.sal AND t2.manager_id > t1.manager_id)
                )
            ORDER BY t2.sal DESC, t2.manager_id DESC
            LIMIT 1
        )
        END as bigger_sal_name
FROM df as t1
ORDER BY vsp_name, sal DESC
"""
sqldf(q)

,manager_name,manager_id,sal,vsp_name,rank,bigger_sal_name
0,Рулон,11,120,Аэропорт,1,-1
1,Олег,12,111,Аэропорт,2,Рулон
2,Настя,9,70,Аэропорт,3,Олег
3,Коля,8,40,Аэропорт,4,Настя
4,Роман,13,40,Аэропорт,4,Настя
5,Виталий,10,10,Аэропорт,5,Роман
6,Дима,14,10,Аэропорт,5,Роман
7,Маша,2,130,Бутово,1,-1
8,Магнус,4,130,Бутово,1,-1
9,Никита,1,120,Бутово,2,Магнус


Еще один способ если нужно вывести все имена коллег с более высокой зарплатой

In [54]:
q = """
WITH t1 as     
    (SELECT manager_name,
        sal,
        manager_id,
        vsp_name,
        DENSE_RANK () OVER (PARTITION BY data_vsp.vsp_name ORDER BY sal DESC) as rank
    FROM data_manager
    LEFT JOIN map_vsp2manager
    ON data_manager.id = map_vsp2manager.manager_id
    LEFT JOIN data_vsp
    ON map_vsp2manager.vsp_id = data_vsp.id)
    
SELECT manager_name,
        sal,
        vsp_name,
        CASE 
        WHEN sal = MAX(sal) OVER (PARTITION BY vsp_name) THEN '-1'
-- способ через ARRAY работает в SQL, но не работает здесь, поэтому здесь через GROUP_CONCAT
--        ELSE (SELECT ARRAY_TO_STRING(ARRAY_AGG(a.manager_name ORDER BY a.manager_id DESC),' ,')
        ELSE (SELECT GROUP_CONCAT(manager_name)
            FROM t1 as a 
            WHERE a.vsp_name = b.vsp_name and a.rank = b.rank-1 
            ORDER BY a.manager_id DESC)
        END as bigger_sal_name,
        CASE 
        WHEN sal = MAX(sal) OVER (PARTITION BY vsp_name) THEN -1
        ELSE (SELECT sal 
            FROM t1 as a 
            WHERE a.vsp_name = b.vsp_name and a.rank = b.rank-1 
            ORDER BY a.manager_id DESC 
            LIMIT 1)
        END as bigger_sal
FROM t1 as b
ORDER BY vsp_name, sal DESC, manager_id DESC"""
sqldf(q)

,manager_name,sal,vsp_name,bigger_sal_name,bigger_sal
0,Рулон,120,Аэропорт,-1,-1
1,Олег,111,Аэропорт,Рулон,120
2,Настя,70,Аэропорт,Олег,111
3,Роман,40,Аэропорт,Настя,70
4,Коля,40,Аэропорт,Настя,70
5,Дима,10,Аэропорт,"Коля,Роман",40
6,Виталий,10,Аэропорт,"Коля,Роман",40
7,Магнус,130,Бутово,-1,-1
8,Маша,130,Бутово,-1,-1
9,Никита,120,Бутово,"Маша,Магнус",130
